In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pydeck as pdk
import osmnx as ox
from module.generate_random_point import *

In [163]:
import toml
import requests
import argparse

### 데이터 읽기

데이터는 택시 승하차 자료에서 여의도 지역만을 추출

In [2]:
def time_transform(data):
    '''
    시간 분까지만 고려 -> 분으로 변환
    '''
    #ride_dtime
    # ride_date = list(map(lambda data: data.day, data['ride_dtime']))
    ride_time = data['ride_dtime'].dt.strftime('%H%M%S')
    ride_time = pd.to_datetime(ride_time, format='%H%M%S')
    
    #alight_dtime
    alight_date = list(map(lambda data: data.day, data['alight_dtime']))
    alight_time = data['alight_dtime'].dt.strftime('%H%M%S')
    alight_time = pd.to_datetime(alight_time, format='%H%M%S')
    
    #시분초 -> 0~1440분으로 변환
    ride_time = ride_time.dt.minute + ride_time.dt.hour*60
    ride_time = ride_time.tolist()
    
    alight_time = alight_time.dt.minute + alight_time.dt.hour*60
    alight_time = alight_time.tolist()
    
    #days = data["ride_dtime"].iloc[0].day    
    #data['ride_dtime'] = [t+1440 if d != days else t for t,d in zip(ride_time, ride_date)]
    #data['alight_dtime'] = [t+1440 if d != days else t for t,d in zip(alight_time, alight_date)]
    data['ride_dtime']  = ride_time
    data['alight_dtime'] = alight_time
    return data

In [3]:
def load_20220408_taxi_data():
    # 20220404(월) ~ 20220410(일)
    taxi_202204 = pd.read_csv('./data/tx_data_202204.csv')
    taxi_202204.columns = ['taxi_id', 'ride_dtime', 'ride_x', 'ride_y', 'alight_dtime', 'alight_x', 'alight_y', 'payment']

    taxi_202204['ride_dtime'] = pd.to_datetime(taxi_202204['ride_dtime'], format='%Y%m%d%H%M%S')
    taxi_202204['alight_dtime'] = pd.to_datetime(taxi_202204['alight_dtime'], format='%Y%m%d%H%M%S')

    # 4월 8일 ~ 15일까지 1주일의 데이터를 불러옴
    taxi_20220408 = taxi_202204.loc[(taxi_202204['ride_dtime'] >= pd.Timestamp(2022,4,8,17)) & (taxi_202204['ride_dtime'] < pd.Timestamp(2022,4,15,20))]
    taxi_20220408 = taxi_20220408.sort_values('ride_dtime').reset_index(drop=False)

    # ride_x, ride_y -> ride_geometry | alight_x, alight_y -> alight_geometry
    taxi_20220408['ride_geometry'] = [Point(x,y) for x,y in zip(taxi_20220408['ride_x'], taxi_20220408['ride_y'])]
    taxi_20220408['alight_geometry'] = [Point(x,y) for x,y in zip(taxi_20220408['alight_x'], taxi_20220408['alight_y'])]

    ### passenger, taxi로 분할
    ## passenger
    passenger = taxi_20220408[['ride_dtime', 'alight_dtime', 'ride_geometry', 'alight_geometry','payment']]
    
    # passenger 2022-04-08 21:00 이상으로 변경
    passenger = passenger.loc[passenger['ride_dtime'] >= pd.Timestamp(2022,4,8,22)]
    passenger = passenger.reset_index(drop=True)
    passenger.rename(columns = {'payment':'real_payment'}, inplace=True)
    # 승객이랑 택시랑 매칭되는 시간
    passenger['dispatch_time'] = 0 

    ### 시간 분단위로 변경    
    passenger = time_transform(passenger)
    passenger_final = passenger.loc[(passenger['ride_dtime']>=1020) & (passenger['ride_dtime']<=1140)] # 오후 5시 ~ 7시 사이의 데이터만 추림
    passenger_final = passenger_final.sort_values('ride_dtime')
    passenger_final.reset_index(inplace=True,drop=True)
    # passenger_final = passenger_final.sort_values('ride_dtime').reset_index(inplace=True,drop=True)
    
    return passenger_final

In [4]:
passenger = load_20220408_taxi_data()

In [173]:
passenger

,ride_dtime,alight_dtime,ride_geometry,alight_geometry,real_payment,dispatch_time
560964,1020,1027,POINT (126.845103 37.496565),POINT (126.853662 37.512899),5300,0
560965,1020,1028,POINT (127.12808 37.551673),POINT (127.133701 37.535086),5000,0
560966,1020,1026,POINT (127.06898 37.666848),POINT (127.065135 37.657018),4100,0
560967,1020,1080,POINT (127.04095 37.641491),POINT (126.965995 37.529045),26400,0
560968,1020,1036,POINT (127.066736 37.531506),POINT (127.04476 37.543066),6700,0
...,...,...,...,...,...,...
1300030,1140,1148,POINT (127.005171 37.506243),POINT (126.988453 37.538797),6200,0
1300031,1140,1190,POINT (127.059916 37.493186),POINT (126.747463 37.494173),40600,0
1300032,1140,1148,POINT (126.913383 37.487228),POINT (126.898172 37.459417),5900,0
1300033,1140,1153,POINT (127.144728 37.493826),POINT (127.139656 37.537246),7700,0


### 여의도 지역 승하차 데이터만 추출

In [5]:
target_places = []

for i in ['여의도동 서울특별시 대한민국']:
    places = ox.geocode_to_gdf([i])
    places = ox.project_gdf(places)
    places['name'] = i
    places = places[['name', 'lat', 'lon', 'geometry']] 
    target_places.append(places)
    
target_places = pd.concat(target_places)
target_places = target_places.to_crs(4326)

In [6]:
def get_data_inside_region(input=passenger, region=target_places):
    ride_points = np.array([passenger.ride_geometry[x].within(target_places.geometry[0]) for x in range(len(passenger))])
    alight_points = np.array([passenger.alight_geometry[x].within(target_places.geometry[0]) for x in range(len(passenger))])
    input = input.loc[(ride_points & alight_points)]
    input.reset_index(drop=True,inplace=True)
    return(input)

In [7]:
passenger_final = get_data_inside_region(input=passenger, region=target_places)

In [229]:
passenger_final.shape # 3시간동안 270건의 주문이 있다고 가정

(270, 10)

In [11]:
passenger_final['O_y'] = [i.y for i in passenger_final['ride_geometry']]
passenger_final['O_x'] = [i.x for i in passenger_final['ride_geometry']]
passenger_final['D_y'] = [i.y for i in passenger_final['alight_geometry']]
passenger_final['D_x'] = [i.x for i in passenger_final['alight_geometry']]

In [227]:
target_layer = pdk.Layer(
    'GeoJsonLayer',
    target_places, 
    opacity=0.2,
    get_fill_color = [255,255,255]
)

pickup_layer = pdk.Layer(
    'ScatterplotLayer',
    passenger_final, 
    get_radius=10,
    get_position='[O_x, O_y]',
    get_color=[0,255,50]
)

dropoff_layer = pdk.Layer(
    'ScatterplotLayer',
    passenger_final, 
    get_radius=10,
    get_position='[D_x, D_y]',
    get_color=[0,50,255]
)


base_map = pdk.Deck(layers=[target_layer,pickup_layer,dropoff_layer],
                      initial_view_state={"latitude": 37.54142, 
                                          "longitude":126.90484,
                                          'zoom':9})

base_map

{"initialViewState": {"latitude": 37.54142, "longitude": 126.90484, "zoom": 9}, "layers": [{"@@type": "GeoJsonLayer", "data": [{"geometry": {"coordinates": [[[126.90483799999998, 37.541416500000004], [126.9079191, 37.5341545], [126.9092778, 37.532254599999995], [126.9106469, 37.5279035], [126.91092040000001, 37.52710269999999], [126.91105140000002, 37.52671900000001], [126.9127731, 37.5240375], [126.9157601, 37.5201548], [126.91723799999998, 37.5193447], [126.917764, 37.51901549999999], [126.91883680000001, 37.51834419999999], [126.9237264, 37.51669880000001], [126.9256133, 37.5160639], [126.9259707, 37.515997000000006], [126.92770610000001, 37.5156722], [126.92866140000001, 37.51554889999999], [126.9316384, 37.51553129999999], [126.9322018, 37.515603399999996], [126.93370449999999, 37.5157956], [126.9361483, 37.51625380000001], [126.9389791, 37.516897799999995], [126.94188010000002, 37.517557700000005], [126.94507930000002, 37.5184564], [126.94886470000002, 37.51966780000001], [126.94993030000002, 37.5265541], [126.94986740000002, 37.5279993], [126.9494006, 37.52892990000001], [126.94921279999998, 37.529304399999994], [126.9485019, 37.5302558], [126.9470356, 37.5312776], [126.9445805, 37.533772299999995], [126.9442738, 37.534163], [126.94421790000001, 37.5342342], [126.9439232, 37.53441730000001], [126.94385529999998, 37.5344591], [126.94382159999999, 37.53445680000001], [126.94370600000002, 37.53444900000001], [126.94365159999998, 37.5345138], [126.9436537, 37.5345232], [126.9436655, 37.5345765], [126.94353560000002, 37.534654999999994], [126.943322, 37.534784200000004], [126.9429439, 37.5350159], [126.9427234, 37.53515099999999], [126.94172290000002, 37.53580879999999], [126.9413883, 37.5360345], [126.9408923, 37.53635029999999], [126.9403439, 37.5366994], [126.93855820000002, 37.53783500000001], [126.93719760000002, 37.538471699999995], [126.93633960000001, 37.53887439999998], [126.9353349, 37.539347199999995], [126.9328984, 37.5405303], [126.9318526, 37.540879], [126.9309549, 37.5411644], [126.9303333, 37.54135569999999], [126.9290194, 37.5402496], [126.92851590000001, 37.53982810000001], [126.92821970000001, 37.539564799999994], [126.92766410000002, 37.5390756], [126.92733490000002, 37.53912770000001], [126.927292, 37.5391345], [126.92727, 37.53912350000001], [126.9272423, 37.53910909999999], [126.9271132, 37.539042], [126.9268737, 37.539086600000005], [126.92678620000001, 37.5391133], [126.92666490000002, 37.53914789999999], [126.92643649999998, 37.5392137], [126.92618520000002, 37.5392898], [126.92586570000002, 37.539468400000004], [126.92557770000002, 37.5395607], [126.9253703, 37.5396272], [126.92536149999998, 37.5396141], [126.92508749999999, 37.5396986], [126.925048, 37.5397118], [126.92485950000001, 37.539773499999995], [126.9248254, 37.53975379999999], [126.92434889999998, 37.5399514], [126.9240438, 37.540074999999995], [126.924042, 37.54007569999999], [126.9235872, 37.540267099999994], [126.9230853, 37.5404526], [126.9228386, 37.5406283], [126.92283240000002, 37.5406284], [126.922424, 37.541429300000004], [126.92180100000002, 37.54142890000001], [126.9171391, 37.5414256], [126.9167664, 37.54142529999999], [126.9159249, 37.541425], [126.90483799999998, 37.541416500000004]]], "type": "Polygon"}, "lat": 37.5284646, "lon": 126.93005217324125, "name": "\uc5ec\uc758\ub3c4\ub3d9 \uc11c\uc6b8\ud2b9\ubcc4\uc2dc \ub300\ud55c\ubbfc\uad6d"}], "getFillColor": [255, 255, 255], "id": "8a5e5ccd-6ae2-4d37-a449-93d0e7bbece4", "opacity": 0.2}, {"@@type": "ScatterplotLayer", "data": [{"D_x": 126.924251, "D_y": 37.523148, "O_x": 126.921438, "O_y": 37.531625, "alight_dtime": 1029, "alight_geometry": {"_Geom": 2404844167040, "_ctypesData": {}, "_isEmpty": false, "_ndim": 2}, "dispatch_time": 0, "real_payment": 7900, "ride_dtime": 1020, "ride_geometry": {"_Geom": 2404772393504, "_ctypesData": {}, "_isEmpty": false, "_ndim": 2}}, {"D_x": 126.917933, "D_y": 37.527696, "O_x": 126.928701, "O_y": 37.526693, "alight_dtime": 1027, "alight_geo

### 간단하게 한번 해보기

In [20]:
# Driver 30명으로 가정
driver_initi_location = create_random_point_based_on_place("여의도동 서울특별시 대한민국", 30)

In [218]:
coordinate_drivers = [[i.y, i.x] for i in driver_initi_location] # 30명의 드라이버에 대한 위치정보

In [226]:
coordinate_objects_start = [[i.y, i.x] for i in passenger_final.ride_geometry.tolist()] # 270개의 물건에 대한 출발위치
coordinate_objects_end =  [[i.y, i.x] for i in passenger_final.alight_geometry.tolist()] # 270개의 물건에 대한 도착위치

In [220]:
coordinates = coordinate_drivers + coordinate_objects[0:2] + coordinate_objects_end[0:2] # 30 + 540: 드라이버와 물건에 대한 모든 위치정보를 담고 있음

In [222]:
drivers = pd.DataFrame({'driver_id': np.arange(30),
                   'location_index': np.arange(30),
                   'time':1020})
                   #'deliveries':np.nan,

drivers = drivers.to_dict('records')

In [223]:
json_data = {
    "coordinates": coordinates,
    "drivers": drivers,
    "shipments": [
        {
            "pickup": {"id": 1001, "location_index": 30}, # 새로운 물건의 출발지
            "delivery": {"id": 1002, "location_index": 32}, # 새로운 물건의 배송지
        },
        {
            "pickup": {"id": 2001, "location_index": 31}, # 새로운 물건의 출발지
            "delivery": {"id": 2002, "location_index": 33}, # 새로운 물건의 배송지
        },
    ],
}


In [224]:
def test(json_data_input = json_data):
    host = "julia.eng.usf.edu"
    port = 4532

    resp = requests.post(f"http://{host}:{port}/pdp", json=json_data_input)
    print("*** Test ***")
    
    j = resp.json()
    print("Raw JSON output: \n", j)

    for route in j:
        print("Driver ID: ", route["driver_id"])
        print("Cost: ", route["cost"])
        for step in route["steps"]:
            print("- Step: ", step)

In [225]:
test(json_data)

*** Test ***
Raw JSON output: 
 [{'cost': 1.6, 'driver_id': 1, 'steps': [{'id': -1, 'location_index': 1, 'type': 'start'}, {'id': 2001, 'location_index': 31, 'type': 'pickup'}, {'id': 2002, 'location_index': 33, 'type': 'delivery'}, {'id': -1, 'location_index': 33, 'type': 'end'}]}, {'cost': 1.28, 'driver_id': 10, 'steps': [{'id': -1, 'location_index': 10, 'type': 'start'}, {'id': 1001, 'location_index': 30, 'type': 'pickup'}, {'id': 1002, 'location_index': 32, 'type': 'delivery'}, {'id': -1, 'location_index': 32, 'type': 'end'}]}]
Driver ID:  1
Cost:  1.6
- Step:  {'id': -1, 'location_index': 1, 'type': 'start'}
- Step:  {'id': 2001, 'location_index': 31, 'type': 'pickup'}
- Step:  {'id': 2002, 'location_index': 33, 'type': 'delivery'}
- Step:  {'id': -1, 'location_index': 33, 'type': 'end'}
Driver ID:  10
Cost:  1.28
- Step:  {'id': -1, 'location_index': 10, 'type': 'start'}
- Step:  {'id': 1001, 'location_index': 30, 'type': 'pickup'}
- Step:  {'id': 1002, 'location_index': 32, 'typ

In [232]:
coordinates[31]

[37.526693, 126.928701]

In [235]:
# passenger_final

,ride_dtime,alight_dtime,ride_geometry,alight_geometry,real_payment,dispatch_time,O_y,O_x,D_y,D_x
0,1020,1029,POINT (126.921438 37.531625),POINT (126.924251 37.523148),7900,0,37.531625,126.921438,37.523148,126.924251
1,1020,1027,POINT (126.928701 37.526693),POINT (126.917933 37.527696),4500,0,37.526693,126.928701,37.527696,126.917933
2,1021,1025,POINT (126.936955 37.51905),POINT (126.929313 37.520295),3800,0,37.519050,126.936955,37.520295,126.929313
3,1021,1033,POINT (126.924506 37.521188),POINT (126.93918 37.520133),8300,0,37.521188,126.924506,37.520133,126.939180
4,1021,1028,POINT (126.914004 37.527157),POINT (126.927712 37.525259),4500,0,37.527157,126.914004,37.525259,126.927712
...,...,...,...,...,...,...,...,...,...,...
265,1139,1149,POINT (126.92881 37.526886),POINT (126.939045 37.520358),5100,0,37.526886,126.928810,37.520358,126.939045
266,1139,1145,POINT (126.926563 37.52585),POINT (126.928453 37.52034),3900,0,37.525850,126.926563,37.520340,126.928453
267,1140,1144,POINT (126.933046 37.526425),POINT (126.931291 37.518946),3800,0,37.526425,126.933046,37.518946,126.931291
268,1140,1147,POINT (126.913701 37.526613),POINT (126.926538 37.52595),4500,0,37.526613,126.913701,37.525950,126.926538
